### **Dual-Simplex Method**

- All the conditions should be less than or equal `<=` inequalities. 
- But, the RHS constant value can be either positive or negative. 
- Essential for sensitivity analysis. 

In [1]:
import os, sys, copy

import numpy as np
from tabulate import tabulate

from helper import visualize_tabulation, all_slack_starting

### Defining the LP Problem

In [7]:
# ==================== DEFINE THE MAIN COMPONENTS OF THE LP PROBLEM ====================
# no. of decision variables (N)
NO_OF_DECISION_VARS = 2

# no. of conditions 
NO_OF_CONDITIONS = 4

# --------------------------------- objective function ---------------------------------
# specify the objective function in the format [<coefficient_1>, <coefficient_2>, ..., <coefficient_N>]
OBJ_FUNC = [-3, -2]

# -------------------------------- type of optimization --------------------------------
OPT_TYPE = "MAX" # or "MIN" 

# ------------------------------------- conditions -------------------------------------
# all the conditions are '<=' inequalities. 
# specify each condition in a new row with the format [<coefficient_1>, <coefficient_2>, ..., <coefficient_N>, <solution>]
conditions = [
    [-1, -1,  -1],
    [ 1,  1,   7],
    [-1, -2, -10],
    [ 0,  1,   3]
]

OBJ_FUNC = np.array(OBJ_FUNC)
conditions = np.array(conditions)

### Creating the Variables and the Initial Tabulation

In [8]:
# find the size of a row
row_size = 1 + NO_OF_DECISION_VARS + NO_OF_CONDITIONS + 1

# find the column size
col_size = 1 + NO_OF_CONDITIONS # no. of basic vars = 1 (for objective function) + no. of conditions

# initialize the initial tabulation
initial_tabulation = np.zeros((col_size, row_size))

# create the variables
var_symbol_arr = []
basic_var_symbol_arr = []

OBJ_FUNC_VAR_SYMBOL = "P"
DECISION_VAR_SYMBOL = "X"
SLACK_VAR_SYMBOL    = "S" # for '<=' inequalities, only slack variables are necessary. 

var_symbol_arr.append(OBJ_FUNC_VAR_SYMBOL)
basic_var_symbol_arr.append(OBJ_FUNC_VAR_SYMBOL)

for i in range(NO_OF_DECISION_VARS):
    var_symbol_arr.append(DECISION_VAR_SYMBOL + str(i+1))

for i in range(NO_OF_CONDITIONS):
    var_symbol_arr.append(SLACK_VAR_SYMBOL + str(i+1))
    basic_var_symbol_arr.append(SLACK_VAR_SYMBOL + str(i+1))

# create the objective function row
initial_tabulation[0][0] = 1
initial_tabulation[0][1 : len(OBJ_FUNC)+1] = -OBJ_FUNC

# create all the other basic variable rows
for i in range(NO_OF_CONDITIONS):

    initial_tabulation[i+1][1 : NO_OF_DECISION_VARS+1] = conditions[i][ :-1]
    initial_tabulation[i+1][-1] = conditions[i][-1] # solution value
    initial_tabulation[i+1][1+NO_OF_DECISION_VARS+i] = 1 # pivot vector

visualize_tabulation(initial_tabulation, all_vars=var_symbol_arr, basic_vars=basic_var_symbol_arr)


┌────┬─────┬──────┬──────┬──────┬──────┬──────┬──────┬───────┐
│    │   P │   X1 │   X2 │   S1 │   S2 │   S3 │   S4 │   sol │
├────┼─────┼──────┼──────┼──────┼──────┼──────┼──────┼───────┤
│ P  │   1 │    3 │    2 │    0 │    0 │    0 │    0 │     0 │
├────┼─────┼──────┼──────┼──────┼──────┼──────┼──────┼───────┤
│ S1 │   0 │   -1 │   -1 │    1 │    0 │    0 │    0 │    -1 │
├────┼─────┼──────┼──────┼──────┼──────┼──────┼──────┼───────┤
│ S2 │   0 │    1 │    1 │    0 │    1 │    0 │    0 │     7 │
├────┼─────┼──────┼──────┼──────┼──────┼──────┼──────┼───────┤
│ S3 │   0 │   -1 │   -2 │    0 │    0 │    1 │    0 │   -10 │
├────┼─────┼──────┼──────┼──────┼──────┼──────┼──────┼───────┤
│ S4 │   0 │    0 │    1 │    0 │    0 │    0 │    1 │     3 │
└────┴─────┴──────┴──────┴──────┴──────┴──────┴──────┴───────┘


### Removing the Negative Values from the Solution Column

In [16]:
tabulation = copy.deepcopy(initial_tabulation)

iteration_no = 0
while (tabulation[:, -1][1: ] < 0).any():
    # until there are negative values in the solution column
    iteration_no += 1
    print("="*20 + f"DUAL SIMPLEX - iteration no.: {iteration_no}" + "="*20)

    # ------------------------------ find the leaving variable ------------------------------
    pivot_row_val = np.min(tabulation[:, -1][1: ]) # max negative solution value
    if list(tabulation[:, -1][1: ]).count(pivot_row_val) > 1:
        print(f"WARNING: there are multiple instances of the maximum negative solution value {pivot_row_val}...")
    pivot_row_idx = list(tabulation[:, -1][1: ]).index(pivot_row_val) + 1
    pivot_row_var = basic_var_symbol_arr[pivot_row_idx]
    print(f"leaving variale : {pivot_row_var}")

    # ----------------------------- find the entering variable ------------------------------
    ratio_row = np.abs(tabulation[0]) / np.abs(tabulation[pivot_row_idx])
    print(f"ratio row       : {ratio_row}")
    pivot_col_val = np.min(ratio_row[1:-1][tabulation[pivot_row_idx][1:-1] < 0]) # surely positive

    # there can be multiple instances of `pivot_col_value`; must find a correct one
    pivot_col_idx = 0
    for _ in range(list(ratio_row[1:-1]).count(pivot_col_val)):
        pivot_col_idx = list(ratio_row[pivot_col_idx+1:-1]).index(pivot_col_val) + 1
        if tabulation[pivot_row_idx][pivot_col_idx] < 0: break
    pivot_col_var = var_symbol_arr[pivot_col_idx]
    print(f"entering variale: {pivot_col_var}")
    basic_var_symbol_arr[pivot_row_idx] = pivot_col_var

    # --------------------- convert the pivot column to a pivot vector ----------------------
    tabulation[pivot_row_idx] /= tabulation[pivot_row_idx][pivot_col_idx]
    pivot_row = tabulation[pivot_row_idx]

    for row_idx in range(tabulation.shape[0]):
        if row_idx == pivot_row_idx: continue

        row = tabulation[row_idx]
        tabulation[row_idx] = row - row[pivot_col_idx] * pivot_row

    visualize_tabulation(tabulation, all_vars=var_symbol_arr, basic_vars=basic_var_symbol_arr)

====================DUAL SIMPLEX - iteration no.: 1====================
leaving variale : S3
ratio row       : [inf  3.  1. nan nan  0. nan  0.]
entering variale: X2
┌────┬─────┬──────┬──────┬──────┬──────┬──────┬──────┬───────┐
│    │   P │   X1 │   X2 │   S1 │   S2 │   S3 │   S4 │   sol │
├────┼─────┼──────┼──────┼──────┼──────┼──────┼──────┼───────┤
│ P  │   1 │  2   │    0 │    0 │    0 │  1   │    0 │   -10 │
├────┼─────┼──────┼──────┼──────┼──────┼──────┼──────┼───────┤
│ S1 │   0 │ -0.5 │    0 │    1 │    0 │ -0.5 │    0 │     4 │
├────┼─────┼──────┼──────┼──────┼──────┼──────┼──────┼───────┤
│ S2 │   0 │  0.5 │    0 │    0 │    1 │  0.5 │    0 │     2 │
├────┼─────┼──────┼──────┼──────┼──────┼──────┼──────┼───────┤
│ X2 │  -0 │  0.5 │    1 │   -0 │   -0 │ -0.5 │   -0 │     5 │
├────┼─────┼──────┼──────┼──────┼──────┼──────┼──────┼───────┤
│ S4 │   0 │ -0.5 │    0 │    0 │    0 │  0.5 │    1 │    -2 │
└────┴─────┴──────┴──────┴──────┴──────┴──────┴──────┴───────┘
===============

/tmp/ipykernel_506091/609116862.py:18: RuntimeWarning: divide by zero encountered in divide
  ratio_row = np.abs(tabulation[0]) / np.abs(tabulation[pivot_row_idx])
/tmp/ipykernel_506091/609116862.py:18: RuntimeWarning: invalid value encountered in divide
  ratio_row = np.abs(tabulation[0]) / np.abs(tabulation[pivot_row_idx])


### Convert Above into a Function

In [14]:
def dual_simplex_to_all_slack_starting(tabulation, all_vars, basic_vars):

    # take a deepcopy of the input tabulation not to alter existing tabulation
    tabulation = copy.deepcopy(tabulation)

    # let's use a loop here...
    for row_idx in range(1, col_size):

        if tabulation[row_idx][-1] < 0:
            # negative solution value -> infeasible
            ratio_row = np.abs(tabulation[0]) / np.abs(tabulation[row_idx])
            min_ratio = np.min(ratio_row[1:-1][tabulation[row_idx][1:-1] < 0])
            if min_ratio == np.inf:
                raise Exception(f"the minimum ratio is infinity; therefore, UNBOUNDED SOLUTIONS EXIST...")

            found = False
            while not found:
                min_ratio_idx = list(ratio_row).index(min_ratio)
                if tabulation[row_idx][min_ratio_idx] < 0: found = True
            tabulation[row_idx] /= tabulation[row_idx][min_ratio_idx]

    visualize_tabulation(tabulation, all_vars=var_symbol_arr, basic_vars=basic_var_symbol_arr)

    return tabulation, all_vars, basic_vars

### Follow-up with All Slack Starting Method

In [17]:
all_slack_starting(tabulation, all_vars=var_symbol_arr, basic_vars=basic_var_symbol_arr, opt_type=OPT_TYPE)

*****there are no more negative values in the objective row; tabulation is optimal...*****

S3 3.0
S4 4.0


(array([[  1.,   0.,   0.,   0.,   0.,   3.,   4., -18.],
        [  0.,   0.,   0.,   1.,   0.,  -1.,  -1.,   6.],
        [  0.,   0.,   0.,   0.,   1.,   1.,   1.,   0.],
        [  0.,   0.,   1.,   0.,   0.,   0.,   1.,   3.],
        [ -0.,   1.,  -0.,  -0.,  -0.,  -1.,  -2.,   4.]]),
 ['P', 'X1', 'X2', 'S1', 'S2', 'S3', 'S4'],
 ['P', 'S1', 'S2', 'X2', 'X1'])

In [11]:
arr = [1, 2, 1]
arr.index(1)

0